# The Battle of Neighborhoods

## 1- Get Paris Arrondissements data

In [1]:
import pandas as pd
import numpy as np

In [2]:
#get NY neighbor json details thx to https://opendata.paris.fr
url = "https://opendata.paris.fr/api/records/1.0/search/?dataset=arrondissements&q=&rows=30"
Paris_Arr_json = pd.read_json(url, orient='index')
Paris_Arr_json.head()

,0
nhits,20
parameters,"{'dataset': 'arrondissements', 'timezone': 'UT..."
records,"[{'datasetid': 'arrondissements', 'recordid': ..."


In [3]:
#get details in 'record'
Paris_Arr_json_records = pd.DataFrame(Paris_Arr_json.loc['records'][0])
Paris_Arr_json_records.head()

,datasetid,recordid,fields,geometry,record_timestamp
0,arrondissements,b8e7cc24b6465feb923772dd142c368be3c19627,"{'n_sq_co': 750001537, 'perimetre': 6054.93686...","{'type': 'Point', 'coordinates': [2.3364433620...",2021-01-03T09:00:32.268000+00:00
1,arrondissements,56b0409d20cad27e65ff4d278425ce3ab12f0078,"{'n_sq_co': 750001537, 'perimetre': 4554.10435...","{'type': 'Point', 'coordinates': [2.3428025468...",2021-01-03T09:00:32.268000+00:00
2,arrondissements,c7af2f6b77a07cc3c13ee3d1b5bfa9114802ea44,"{'n_sq_co': 750001537, 'perimetre': 11253.1824...","{'type': 'Point', 'coordinates': [2.3848209601...",2021-01-03T09:00:32.268000+00:00
3,arrondissements,ab21ad6cc851338444156f8276d3ad1c5b2e84c1,"{'n_sq_co': 750001537, 'perimetre': 8282.01188...","{'type': 'Point', 'coordinates': [2.3800583082...",2021-01-03T09:00:32.268000+00:00
4,arrondissements,539d0abc949b6ad6f8805f808fb1935324f77ccd,"{'n_sq_co': 750001537, 'perimetre': 6739.37505...","{'type': 'Point', 'coordinates': [2.3607284878...",2021-01-03T09:00:32.268000+00:00


In [4]:
#get details in 'fields'
Paris_Arr_json_fields = pd.DataFrame(pd.json_normalize(Paris_Arr_json_records['fields']))
Paris_Arr_json_fields.head()

,n_sq_co,perimetre,l_ar,surface,geom_x_y,n_sq_ar,l_aroff,c_arinsee,c_ar,geom.type,geom.coordinates
0,750001537,6054.936862,1er Ardt,1.824613e+06,"[48.8625627018, 2.33644336205]",750000001,Louvre,75101,1,Polygon,"[[[2.328007329038849, 48.86991742140716], [2.3..."
1,750001537,4554.104360,2ème Ardt,9.911537e+05,"[48.8682792225, 2.34280254689]",750000002,Bourse,75102,2,Polygon,"[[[2.351518483670822, 48.8644258050741], [2.35..."
2,750001537,11253.182479,19ème Ardt,6.792651e+06,"[48.8870759966, 2.38482096015]",750000019,Buttes-Chaumont,75119,19,Polygon,"[[[2.38942869613216, 48.901218118234496], [2.3..."
3,750001537,8282.011886,11ème Ardt,3.665442e+06,"[48.8590592213, 2.3800583082]",750000011,Popincourt,75111,11,Polygon,"[[[2.3962365763098292, 48.85415458748718], [2...."
4,750001537,6739.375055,10ème Ardt,2.891739e+06,"[48.8761300365, 2.36072848785]",750000010,Entrepôt,75110,10,Polygon,"[[[2.364685889119478, 48.88436904340793], [2.3..."


In [5]:
#Select only usefull data
df_Paris_Arr = Paris_Arr_json_fields[['l_ar','l_aroff','surface','geom_x_y']]
df_Paris_Arr = df_Paris_Arr.rename(columns={'l_ar':'Num Arrondissement','l_aroff':'Nom','surface':'Surface','geom_x_y':'Coordinates'})
print(df_Paris_Arr.shape)
df_Paris_Arr.head()

(20, 4)


,Num Arrondissement,Nom,Surface,Coordinates
0,1er Ardt,Louvre,1.824613e+06,"[48.8625627018, 2.33644336205]"
1,2ème Ardt,Bourse,9.911537e+05,"[48.8682792225, 2.34280254689]"
2,19ème Ardt,Buttes-Chaumont,6.792651e+06,"[48.8870759966, 2.38482096015]"
3,11ème Ardt,Popincourt,3.665442e+06,"[48.8590592213, 2.3800583082]"
4,10ème Ardt,Entrepôt,2.891739e+06,"[48.8761300365, 2.36072848785]"


In [6]:
import folium # map rendering library

map_paris = folium.Map(location=[ 48.8625627018, 2.33644336205], zoom_start=12)

# add markers to map
for lat_lng, label in zip(df_Paris_Arr['Coordinates'], df_Paris_Arr['Num Arrondissement']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        lat_lng,
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_paris) 

map_paris

## 2- Get Paris Venues from Foursquare

In [7]:
#Foursquare credentials
CLIENT_ID = 'UP5SMBXQ1LN3OQCTMH423QPSUAQRZ5L2QTLLSY0QKDGFNAWN' # your Foursquare ID
CLIENT_SECRET = 'TCQRSW2DNSWQWJLPJQJAT2RX0QYFVJRKEZYEBBWOGTWHRFH3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UP5SMBXQ1LN3OQCTMH423QPSUAQRZ5L2QTLLSY0QKDGFNAWN
CLIENT_SECRET:TCQRSW2DNSWQWJLPJQJAT2RX0QYFVJRKEZYEBBWOGTWHRFH3


In [8]:
#Get venues function
import requests # library to handle requests

def getNearbyVenues(names, lat_lon, radius=500):
    
    venues_list=[]
    for name, lat_lon in zip(names, lat_lon):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat_lon[0], 
            lat_lon[1], 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat_lon[0], 
            lat_lon[1], 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Arrondissement', 
                  'Arrondissement Latitude', 
                  'Arrondissement Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
#Build Paris Venues
paris_venues = getNearbyVenues(names=df_Paris_Arr['Num Arrondissement'], lat_lon=df_Paris_Arr['Coordinates'])

1er Ardt
2ème Ardt
19ème Ardt
11ème Ardt
10ème Ardt
17ème Ardt
20ème Ardt
15ème Ardt
12ème Ardt
4ème Ardt
14ème Ardt
13ème Ardt
6ème Ardt
5ème Ardt
8ème Ardt
16ème Ardt
9ème Ardt
18ème Ardt
3ème Ardt
7ème Ardt


In [10]:
#Check Result
print(paris_venues.shape)
paris_venues.head()

(1276, 7)


,Arrondissement,Arrondissement Latitude,Arrondissement Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,1er Ardt,48.862563,2.336443,Musée du Louvre,48.860847,2.336440,Art Museum
1,1er Ardt,48.862563,2.336443,Palais Royal,48.863236,2.337127,Historic Site
2,1er Ardt,48.862563,2.336443,Comédie-Française,48.863088,2.336612,Theater
3,1er Ardt,48.862563,2.336443,Cour Napoléon,48.861172,2.335088,Plaza
4,1er Ardt,48.862563,2.336443,Place du Palais Royal,48.862523,2.336688,Plaza


## 3- Explore Paris Venues

In [11]:
# one hot encoding
paris_venues_by_arr = pd.get_dummies(paris_venues[['Venue Category']], prefix="", prefix_sep="")

# add Arrondissement column back to dataframe
paris_venues_by_arr['Arrondissement'] = paris_venues['Arrondissement'] 

# move Arrondissement column to the first column
fixed_columns = [paris_venues_by_arr.columns[-1]] + list(paris_venues_by_arr.columns[:-1])
paris_onehot = paris_venues_by_arr[fixed_columns]

#Group by Arrondissement
paris_venues_by_arr = paris_venues_by_arr.groupby('Arrondissement').mean().reset_index()
paris_venues_by_arr

,Arrondissement,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Waterfront,Wine Bar,Wine Shop,Women's Store,Zoo,Zoo Exhibit
0,10ème Ardt,0.000000,0.02,0.000000,0.00,0.00000,0.000000,0.000000,0.000000,0.020000,...,0.010000,0.00,0.000000,0.000000,0.00,0.010000,0.020000,0.000000,0.00,0.00
1,11ème Ardt,0.014493,0.00,0.000000,0.00,0.00000,0.000000,0.014493,0.000000,0.014493,...,0.014493,0.00,0.014493,0.043478,0.00,0.028986,0.000000,0.014493,0.00,0.00
2,12ème Ardt,0.000000,0.00,0.000000,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.25,0.25
3,13ème Ardt,0.000000,0.00,0.000000,0.00,0.00000,0.000000,0.000000,0.000000,0.213115,...,0.000000,0.00,0.000000,0.213115,0.00,0.000000,0.000000,0.000000,0.00,0.00
4,14ème Ardt,0.000000,0.00,0.000000,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00
5,15ème Ardt,0.000000,0.00,0.000000,0.00,0.00000,0.000000,0.000000,0.016393,0.000000,...,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00
6,16ème Ardt,0.000000,0.00,0.000000,0.00,0.00000,0.000000,0.125000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00
7,17ème Ardt,0.000000,0.00,0.000000,0.00,0.00000,0.000000,0.016393,0.000000,0.016393,...,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00
8,18ème Ardt,0.000000,0.00,0.000000,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.022727,0.00,0.022727,0.000000,0.000000,0.00,0.00
9,19ème Ardt,0.000000,0.00,0.023256,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.023256,0.00,0.000000,0.023256,0.000000,0.00,0.00


In [12]:
#function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [13]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Arrondissement']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Arrondissement'] = paris_venues_by_arr['Arrondissement']

for ind in np.arange(paris_venues_by_arr.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(paris_venues_by_arr.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Arrondissement,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,10ème Ardt,French Restaurant,Bistro,Hotel,Indian Restaurant,Café,Coffee Shop,Pizza Place,Thai Restaurant,Mediterranean Restaurant,Fast Food Restaurant
1,11ème Ardt,French Restaurant,Café,Restaurant,Supermarket,Vietnamese Restaurant,Pastry Shop,Bakery,Italian Restaurant,Japanese Restaurant,Pizza Place
2,12ème Ardt,Zoo Exhibit,Supermarket,Monument / Landmark,Zoo,Corsican Restaurant,Cosmetics Shop,Food & Drink Shop,Flower Shop,Fish & Chips Shop,Fast Food Restaurant
3,13ème Ardt,Vietnamese Restaurant,Asian Restaurant,Chinese Restaurant,Thai Restaurant,French Restaurant,Juice Bar,Coffee Shop,Cosmetics Shop,Butcher,Cambodian Restaurant
4,14ème Ardt,French Restaurant,Hotel,Bistro,Laundromat,Brasserie,Food & Drink Shop,Plaza,Sushi Restaurant,Organic Grocery,Bakery
5,15ème Ardt,Hotel,Italian Restaurant,French Restaurant,Plaza,Lebanese Restaurant,Supermarket,Coffee Shop,Thai Restaurant,Bakery,Park
6,16ème Ardt,Lake,French Restaurant,Plaza,Boat or Ferry,Art Museum,Bus Station,Park,Electronics Store,Fish & Chips Shop,Fast Food Restaurant
7,17ème Ardt,French Restaurant,Hotel,Italian Restaurant,Bakery,Japanese Restaurant,Restaurant,Café,Bistro,Plaza,Metro Station
8,18ème Ardt,Bar,French Restaurant,Hotel,Café,Restaurant,Convenience Store,Park,Sandwich Place,Cheese Shop,Plaza
9,19ème Ardt,Bar,French Restaurant,Supermarket,Brewery,Bistro,Beer Bar,Seafood Restaurant,Hotel,Farmers Market,Metro Station


## 4- Clustering

In [14]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 8

paris_grouped_clustering = paris_venues_by_arr.drop('Arrondissement', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(paris_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 7, 2, 1, 6, 7, 4, 3, 5, 5])

In [15]:
# add clustering labels
try : 
    neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
except:
    None
    
paris_merged = df_Paris_Arr
paris_merged = paris_merged.join(neighborhoods_venues_sorted.set_index('Arrondissement'), on='Num Arrondissement')

paris_merged.head() # check the last columns!

,Num Arrondissement,Nom,Surface,Coordinates,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1er Ardt,Louvre,1.824613e+06,"[48.8625627018, 2.33644336205]",7,French Restaurant,Plaza,Japanese Restaurant,Hotel,Art Museum,Italian Restaurant,Coffee Shop,Café,Historic Site,Cheese Shop
1,2ème Ardt,Bourse,9.911537e+05,"[48.8682792225, 2.34280254689]",0,French Restaurant,Wine Bar,Cocktail Bar,Bistro,Creperie,Bakery,Hotel,Italian Restaurant,Cheese Shop,Concert Hall
2,19ème Ardt,Buttes-Chaumont,6.792651e+06,"[48.8870759966, 2.38482096015]",5,Bar,French Restaurant,Supermarket,Brewery,Bistro,Beer Bar,Seafood Restaurant,Hotel,Farmers Market,Metro Station
3,11ème Ardt,Popincourt,3.665442e+06,"[48.8590592213, 2.3800583082]",7,French Restaurant,Café,Restaurant,Supermarket,Vietnamese Restaurant,Pastry Shop,Bakery,Italian Restaurant,Japanese Restaurant,Pizza Place
4,10ème Ardt,Entrepôt,2.891739e+06,"[48.8761300365, 2.36072848785]",0,French Restaurant,Bistro,Hotel,Indian Restaurant,Café,Coffee Shop,Pizza Place,Thai Restaurant,Mediterranean Restaurant,Fast Food Restaurant


In [16]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[ 48.8625627018, 2.33644336205], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat_lon, poi, cluster in zip(paris_merged['Coordinates'], paris_merged['Num Arrondissement'], paris_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        lat_lon,
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster 1

In [17]:
i=0
paris_merged.loc[paris_merged['Cluster Labels'] == i, paris_merged.columns[[0,1] + list(range(5, paris_merged.shape[1]))]]

,Num Arrondissement,Nom,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,2ème Ardt,Bourse,French Restaurant,Wine Bar,Cocktail Bar,Bistro,Creperie,Bakery,Hotel,Italian Restaurant,Cheese Shop,Concert Hall
4,10ème Ardt,Entrepôt,French Restaurant,Bistro,Hotel,Indian Restaurant,Café,Coffee Shop,Pizza Place,Thai Restaurant,Mediterranean Restaurant,Fast Food Restaurant
9,4ème Ardt,Hôtel-de-Ville,French Restaurant,Ice Cream Shop,Hotel,Clothing Store,Pedestrian Plaza,Pastry Shop,Wine Bar,Plaza,Park,Coffee Shop
13,5ème Ardt,Panthéon,French Restaurant,Italian Restaurant,Science Museum,Hotel,Plaza,Pub,Bakery,Café,Coffee Shop,Bar
16,9ème Ardt,Opéra,French Restaurant,Hotel,Bakery,Bistro,Cocktail Bar,Wine Bar,Lounge,Japanese Restaurant,Pizza Place,Tea Room


### Cluster 2

In [18]:
i=1
paris_merged.loc[paris_merged['Cluster Labels'] == i, paris_merged.columns[[0,1] + list(range(5, paris_merged.shape[1]))]]

,Num Arrondissement,Nom,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,13ème Ardt,Gobelins,Vietnamese Restaurant,Asian Restaurant,Chinese Restaurant,Thai Restaurant,French Restaurant,Juice Bar,Coffee Shop,Cosmetics Shop,Butcher,Cambodian Restaurant


### Cluster 3

In [19]:
i=2
paris_merged.loc[paris_merged['Cluster Labels'] == i, paris_merged.columns[[0,1] + list(range(5, paris_merged.shape[1]))]]

,Num Arrondissement,Nom,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,12ème Ardt,Reuilly,Zoo Exhibit,Supermarket,Monument / Landmark,Zoo,Corsican Restaurant,Cosmetics Shop,Food & Drink Shop,Flower Shop,Fish & Chips Shop,Fast Food Restaurant


### Cluster 4

In [20]:
i=3
paris_merged.loc[paris_merged['Cluster Labels'] == i, paris_merged.columns[[0,1] + list(range(5, paris_merged.shape[1]))]]

,Num Arrondissement,Nom,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,17ème Ardt,Batignolles-Monceau,French Restaurant,Hotel,Italian Restaurant,Bakery,Japanese Restaurant,Restaurant,Café,Bistro,Plaza,Metro Station
14,8ème Ardt,Élysée,French Restaurant,Hotel,Spa,Department Store,Resort,Hotel Bar,Plaza,Corsican Restaurant,Park,Modern European Restaurant
19,7ème Ardt,Palais-Bourbon,Hotel,French Restaurant,Café,Italian Restaurant,Plaza,History Museum,Cocktail Bar,Art Museum,Dessert Shop,Cheese Shop


### Cluster 5

In [21]:
i=4
paris_merged.loc[paris_merged['Cluster Labels'] == i, paris_merged.columns[[0,1] + list(range(5, paris_merged.shape[1]))]]

,Num Arrondissement,Nom,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,16ème Ardt,Passy,Lake,French Restaurant,Plaza,Boat or Ferry,Art Museum,Bus Station,Park,Electronics Store,Fish & Chips Shop,Fast Food Restaurant


In [22]:
i=5
paris_merged.loc[paris_merged['Cluster Labels'] == i, paris_merged.columns[[0,1] + list(range(5, paris_merged.shape[1]))]]

,Num Arrondissement,Nom,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,19ème Ardt,Buttes-Chaumont,Bar,French Restaurant,Supermarket,Brewery,Bistro,Beer Bar,Seafood Restaurant,Hotel,Farmers Market,Metro Station
17,18ème Ardt,Buttes-Montmartre,Bar,French Restaurant,Hotel,Café,Restaurant,Convenience Store,Park,Sandwich Place,Cheese Shop,Plaza


In [23]:
i=6
paris_merged.loc[paris_merged['Cluster Labels'] == i, paris_merged.columns[[0,1] + list(range(5, paris_merged.shape[1]))]]

,Num Arrondissement,Nom,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,14ème Ardt,Observatoire,French Restaurant,Hotel,Bistro,Laundromat,Brasserie,Food & Drink Shop,Plaza,Sushi Restaurant,Organic Grocery,Bakery


In [24]:
i=7
paris_merged.loc[paris_merged['Cluster Labels'] == i, paris_merged.columns[[0,1] + list(range(5, paris_merged.shape[1]))]]

,Num Arrondissement,Nom,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1er Ardt,Louvre,French Restaurant,Plaza,Japanese Restaurant,Hotel,Art Museum,Italian Restaurant,Coffee Shop,Café,Historic Site,Cheese Shop
3,11ème Ardt,Popincourt,French Restaurant,Café,Restaurant,Supermarket,Vietnamese Restaurant,Pastry Shop,Bakery,Italian Restaurant,Japanese Restaurant,Pizza Place
6,20ème Ardt,Ménilmontant,Bakery,Japanese Restaurant,Plaza,French Restaurant,Bistro,Bar,Italian Restaurant,Restaurant,Café,Park
7,15ème Ardt,Vaugirard,Hotel,Italian Restaurant,French Restaurant,Plaza,Lebanese Restaurant,Supermarket,Coffee Shop,Thai Restaurant,Bakery,Park
12,6ème Ardt,Luxembourg,Plaza,Pastry Shop,French Restaurant,Chocolate Shop,Bakery,Tea Room,Fountain,Wine Bar,Cocktail Bar,Bistro
18,3ème Ardt,Temple,French Restaurant,Japanese Restaurant,Gourmet Shop,Bakery,Art Gallery,Italian Restaurant,Wine Bar,Sandwich Place,Cocktail Bar,Coffee Shop
